In [1]:
from census import Census
from config import (census_key)
from us import states
import pandas as pd


c = Census(census_key, year=2013)

In [2]:
#c.sf1.state_county_tract('NAME', states.TX.fips, '113', Census.ALL)

In [3]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E", "B23025_005E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Count"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Count
0,08518,5217.0,41.5,74286.0,33963.0,170.0,3.258578,321.0
1,08520,27468.0,37.4,90293.0,37175.0,1834.0,6.676860,1418.0
2,08525,4782.0,47.1,118656.0,59848.0,43.0,0.899205,138.0
3,08527,54867.0,42.2,88588.0,37021.0,2191.0,3.993293,2910.0
4,08528,245.0,48.5,58676.0,49117.0,0.0,0.000000,0.0


In [4]:
dallasZips = pd.read_csv("../Resources/DallasZipCodes.csv")
dallasZips.head()

FileNotFoundError: [Errno 2] File b'../Resources/DallasZipCodes.csv' does not exist: b'../Resources/DallasZipCodes.csv'

In [5]:
print(census_pd.dtypes)
print(dallasZips.dtypes)

Zipcode                object
Population            float64
Median Age            float64
Household Income      float64
Per Capita Income     float64
Poverty Count         float64
Poverty Rate          float64
Unemployment Count    float64
dtype: object


NameError: name 'dallasZips' is not defined

In [6]:
census_pd["Zipcode"] = census_pd["Zipcode"].astype(int)
mask = [zipcode in dallasZips.ZipCode.values for zipcode in census_pd.Zipcode]
dallasSub = census_pd[mask]

NameError: name 'dallasZips' is not defined

In [ ]:
dallasSub.head()

In [7]:
dallasSubMerge = pd.merge(dallasSub,dallasZips[['ZipCode','City']],left_on='Zipcode', right_on='ZipCode', how='inner')
dallasSubMerge = dallasSubMerge.drop(columns=["ZipCode"])
dallasSubMerge

NameError: name 'dallasSub' is not defined

In [ ]:
#drop NaN values
DroppedData = dallasSubMerge.dropna(how="any")


DroppedData.describe()

In [22]:
dallasSubMerge.to_csv("../Resources/DallasCensusData.csv", index=False)